# 1000x faster data manipulation!
## Vectorizing with pandas and NumPy
- Here we will see the power of vectorization.
- We will se how to speedup the data manipulation in pandas.
- we will se some important functions
- we will learn about `np.where()` `np.select` `vectorizer`.

In [2]:
import pandas as pd
import numpy as np
import re
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# pd.set_option('max_columns', 15)
# pd.set_option('chained_assignment', None)

## Read in and setup the data 
This CSV is an example of the data I worked on while practicing learning how to vectorize former `.apply` functions. The data here have been scrubbed and scrambled to not relate to any real-life data.

In [3]:
df = pd.read_csv('sample_data.csv', 
                 parse_dates=['Date Created', 'Original Record: Date Created'])

### Data inspection 

In [4]:
df.shape
df.head(5)

(273399, 14)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,Original Record: Date Created,Date Created,Inactive,Specialty,Providers,duplicate_leads,bad_leads
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,False,True
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,True,True
2,16,6936,Practice,Nonpaid : Webinar : AdvancedFax and Patient : ...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-12-03 17:33:00,No,Dermatology,0.0,False,True
3,16,4557,Practice,Nonpaid : Webinar : AdvancedFax and Patient : ...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-12-03 17:33:00,No,Dermatology,0.0,True,True
4,20,8637,Practice,Nonpaid : Webinar : HIPAA Audit Prep : LSID 53...,- None -,CLIENT-Live,CLIENT-Live,2003-04-30 18:15:00,2016-08-24 14:09:00,No,Plastic Surgery,1.0,False,False


In [4]:
df.dtypes

Internal ID                               int64
ID                                        int64
Category                                 object
Lead Source                              object
Lead Source Category                     object
Current Status                           object
Status at Time of Lead                   object
Original Record: Date Created    datetime64[ns]
Date Created                     datetime64[ns]
Inactive                                 object
Specialty                                object
Providers                               float64
duplicate_leads                            bool
bad_leads                                  bool
dtype: object

#### check how much memory dataframe is taking.

In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273399 entries, 0 to 273398
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Internal ID                    273399 non-null  int64         
 1   ID                             273399 non-null  int64         
 2   Category                       273399 non-null  object        
 3   Lead Source                    273399 non-null  object        
 4   Lead Source Category           273399 non-null  object        
 5   Current Status                 273399 non-null  object        
 6   Status at Time of Lead         273399 non-null  object        
 7   Original Record: Date Created  273399 non-null  datetime64[ns]
 8   Date Created                   273399 non-null  datetime64[ns]
 9   Inactive                       273399 non-null  object        
 10  Specialty                      273399 non-null  object        
 11  

# First attempt at vectorizing with conditionals

In [6]:
def set_lead_status(row):
    if row['Current Status'] == '- None -':
        return row['Status at Time of Lead']
    else:
        return row['Current Status']

In [13]:
# df['Current Status'].value_counts()
# df['Current Status'].isna().sum()

In [14]:
%%timeit
test = df.apply(set_lead_status, axis=1)

3.73 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
# Or another way to do it...
start = time.time()

test = df.apply(set_lead_status, axis=1)

end = time.time()
print(round(end - start, 2))

5.03


In the above examples you can see we have loop each row of dataframe and then check the corresponding condition. To avoid this rowwise looping let's try to pass complete series as args instead of row.

 **Let's See What Will Happen:**

In [17]:
def set_lead_status(col1, col2):
    if col1 == '- None -':
        return col2
    else:
        return col1

In [18]:
test1 = set_lead_status(df['Current Status'], df['Status at Time of Lead'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# What Is `numpy.where()`
[Documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html)
Similar to Excel 'IF' function, you give it a condition, except this condition can handle the truthiness for the entire array/column. Then give it what to return if elements on that boolean array are true or false.

In [19]:
%%timeit
# Pandas Series Vectorized baby!!

# you can pass the output directly into a pandas Series

df['normalized_status'] = np.where(
    df['Current Status'] == '- None -',   # <-- condition
    df['Status at Time of Lead'],         # <-- return if true
    df['Current Status']                  # <-- return if false
)

21.2 ms ± 638 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
df.head(2)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,Original Record: Date Created,Date Created,Inactive,Specialty,Providers,duplicate_leads,bad_leads,normalized_status
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,False,True,CLIENT-Deactivated
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,True,True,CLIENT-Deactivated


In [22]:
print(f"`np.where` is {round((5.03 * 1000) / 21.2, 1)}x faster than `.apply`")

`np.where` is 237.3x faster than `.apply`


Sofia mentions in her talk that you can go even faster by accessing the underlying NumPy arrays from your pandas series. This makes it faster b/c now there's only a NumPy array of your data to pass to C, with no need for handling all the stuff attached to a pandas Series that makes them so convenient to work with.

In [26]:
type(df['Status at Time of Lead'].values)

numpy.ndarray

In [28]:
%%timeit
# NumPy Vectorized baby!!

df['normalized_status'] = np.where(
    df['Current Status'].values == '- None -',
    df['Status at Time of Lead'].values, 
    df['Current Status'].values
)

8.69 ms ± 541 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
print(f"`np.where` w/ numpy vectorization is {round((5.03 * 1000) / 8.69, 1)}x faster than `.apply`")

`np.where` w/ numpy vectorization is 578.8x faster than `.apply`


In [ ]:
# %%timeit
# test = df.apply(works_but_slow, axis=1, raw=True)  # There is a significant speed improvement from using raw=True 
# # with pd.DataFrame.apply versus without. This option feeds NumPy arrays to the custom function instead of pd.Series objects.
# # https://stackoverflow.com/questions/52673285/performance-of-pandas-apply-vs-np-vectorize-to-create-new-column-from-existing-c

# What about `numpy.vectorize()`?

This is a function that will take a Python function and turn it into a NumPy ufunc, so it can handle vectorized approaches. It _vectorizes_ your function, not necessarily how that function applies to your data. Big difference there.

Resources:
 - https://docs.scipy.org/doc/numpy/reference/generated/numpy.vectorize.html
 - https://www.experfy.com/blog/why-you-should-forget-loops-and-embrace-vectorization-for-data-science


In [30]:
# Here is our previous function that I tried to vectorize but couldn't due to the ValueError. 
def works_fast_maybe(col1, col2):
    if col1 == '- None -':
        return col2
    else:
        return col1

In [31]:
# with the np.vectorize method --> returns a vectorized callable
vectfunc = np.vectorize(works_fast_maybe) #otypes=[np.float],cache=False)

In [36]:
%%timeit 
list(vectfunc(df['Status at Time of Lead'], df['Current Status']))

193 ms ± 5.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Some guy on Medium thought this was faster -- using index setting -- but it turns out it's actually not vectorizing

In [37]:
def can_I_go_any_faster(status_at_time, current_status):
    # this works fine if you're setting static values
    df['test'] = 'test'# status_at_time
    df.loc[status_at_time == '- None -', 'test'] = current_status  # <-- ValueError, indexes don't match!
    df.loc[status_at_time != '- None -', 'test'] = status_at_time

In [38]:
%%timeit
test4 = can_I_go_any_faster(df['Status at Time of Lead'], df['Current Status'])

54.8 ms ± 3.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
df.head(2)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,Original Record: Date Created,Date Created,Inactive,Specialty,Providers,duplicate_leads,bad_leads,normalized_status,test
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,False,True,CLIENT-Deactivated,CLIENT-Deactivated
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,True,True,CLIENT-Deactivated,CLIENT-Deactivated


In [42]:
def can_I_go_any_faster2(status_at_time, current_status):
    # this works fine if you're setting static values
    df['test'] = 'test'# status_at_time
    df.loc[status_at_time == '- None -', 'test'] = 'statys1_isNone' 
    df.loc[status_at_time != '- None -', 'test'] = 'statys2_notNone'

In [43]:
%%timeit
test5 = can_I_go_any_faster2(df['Status at Time of Lead'], df['Current Status'])

45.3 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Multiple conditions

### lead_category

In [46]:
list1 = ['LEAD-3 Flame No Contact', 'LEAD-Campaign', 'LEAD-Claim', 'LEAD-Contact Program', 
         'LEAD-General Pool', 'LEAD-No Contact', 'LEAD-Nurture', 'LEAD-Unqualified', 'PROSPECT-LOST']

list2 = ['- None -', 'CLIENT-Closed-Sold', 'CLIENT-Handoff', 'CLIENT-Implementation', 'CLIENT-Implementation (EMR)',
         'CLIENT-Live', 'CLIENT-Partner', 'CLIENT-Referring Consultant', 'CLIENT-Transferred', 'LEAD-Engaged', 
         'LEAD-Long-Term Opportunity', 'PROSPECT-CURRENT', 'PROSPECT-LONG TERM', 'PROSPECT-NO DECISION']

# apply version
def lead_category(row):
    if row['Original Record: Date Created'] == row['Date Created']:
        return 'New Lead'
    elif row['normalized_status'].startswith('CLI'):
        return 'Client Lead'
    elif row['normalized_status'] in list1:
        return 'MTouch Lead'
    elif row['normalized_status'] in list2:
        return 'EMTouch Lead'
    return 'NA'

In [47]:
%%timeit
df['lead_category0'] = df.apply(lead_category, axis=1)

4.82 s ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


You can call a `np.where` for every condition, and it will run fine. But it gets a little hard to read after a while.

In [48]:
%%timeit
df['lead_category'] = \
    np.where(df['Original Record: Date Created'].values == df['Date Created'].values, 'New Lead', 
            np.where(df['normalized_status'].str.startswith('CLI').values, 'Client Lead', 
                    np.where(df['normalized_status'].isin(list1).values, 'MTouch Lead', 
                            np.where(df['normalized_status'].isin(list2).values, 'EMTouch Lead', 
                                     'NA') 
                                  )
                         )
                )

187 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# What Is `numpy.select()`
Cleaner (and even faster!) and doing multiple nested `np.where` calls for each conditions.
Order of operations matter!

[Documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html)

In [49]:
%%timeit
conditions = [
    df['Original Record: Date Created'].values == df['Date Created'].values,
    df['normalized_status'].str.startswith('CLI').values,
    df['normalized_status'].isin(list1).values,
    df['normalized_status'].isin(list2).values
]

choices = [
    'New Lead', 
    'Client Lead', 
    'MTouch Lead',
    'EMTouch Lead'
]


df['lead_category1'] = np.select(conditions, choices, default='NA')  # Order of operations matter!

171 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
# Their output logic is the same
(df.lead_category == df.lead_category1).all()

True

In [53]:
print(f"`np.select` w/ numpy vectorization is {round((4.82 * 1000) / 187, 2)}x faster than nested .apply()")

`np.select` w/ numpy vectorization is 25.78x faster than nested .apply()


In [54]:
print(f"`np.select` is {round(187 / 171, 2)} faster than nested `np.where`")

`np.select` is 1.09 faster than nested `np.where`


## What about nested multiple conditionals? Can we vectorize that?
Yes!

In [55]:
# This is something you might think you can't vectorize, but you sure can!
def sub_conditional(row):
    if row['Inactive'] == 'No':
        if row['Providers'] == 0:
            return 'active_no_providers'
        elif row['Providers'] < 5:
            return 'active_small'
        else:
            return 'active_normal'
    elif row['duplicate_leads']:
        return 'is_dup'
    else:
        if row['bad_leads']:
            return 'active_bad'
        else:
            return 'active_good'

In [56]:
%%timeit
# Let's time how long it takes to apply a nested multiple condition func
df['lead_type'] = df.apply(sub_conditional, axis=1)

4.66 s ± 216 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%%timeit

# With np.select, could do .values here for additional speed, but left out to avoid too much text
conditions = [
    ((df['Inactive'] == 'No') & (df['Providers'] == 0)),
    ((df['Inactive'] == 'No') & (df['Providers'] < 5)),
    df['Inactive'] == 'No',
    df['duplicate_leads'],  # <-- you can also just evaluate boolean arrays
    df['bad_leads'],
]

choices = [
    'active_no_providers',
    'active_small',
    'active_normal',
    'is_dup',
    'active_bad',
]

df['lead_type_vec'] = np.select(conditions, choices, default='NA')

120 ms ± 4.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
mask = (
    ((df['Inactive'] == 'No') & (df['Providers'] == 0))
    & ((df['Inactive'] == 'No') & (df['Providers'] < 5))
    & (df['Inactive'] == 'No')
    & (df['duplicate_leads'])  # <-- you can also just evaluate boolean arrays
    & df['bad_leads']
)





In [58]:
print(f"`np.select` is {round((4.66* 1000) / 120, 2)} faster than nested .apply()")

`np.select` is 38.83 faster than nested .apply()


In [33]:
%%timeit

# With np.select
conditions = [
    ((df['Inactive'].values == 'No') & (df['Providers'].values == 0)),
    ((df['Inactive'].values == 'No') & (df['Providers'].values < 5)),
    df['Inactive'].values == 'No',
    df['duplicate_leads'].values,  # <-- you can also just evaluate boolean arrays
    df['bad_leads'].values,
]

choices = [
    'active_no_providers',
    'active_small',
    'active_normal',
    'is_dup',
    'active_bad',
]

df['lead_type_vec'] = np.select(conditions, choices, default='NA')

/home/nathancheever/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.


35.8 ms ± 257 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
print(f"`np.select` w/ vectorization is {round((4.66 * 1000) / 35.8, 2)} faster than nested .apply()")

`np.select` w/ vectorization is 130.17 faster than nested .apply()


# What about more complicated things?
Of course this is just a tiny sample of things you might encounter, but I thought they could be useful to see how vectorization can still apply even with otherwise difficult cases.

#### Strings

In [35]:
df.head(2)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,...,normalized_status,test,lead_category0,lead_category,lead_category1,lead_type,lead_type_vec
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,...,CLIENT-Deactivated,statys1_isNone,Client Lead,Client Lead,Client Lead,active_no_providers,active_no_providers
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,...,CLIENT-Deactivated,statys1_isNone,Client Lead,Client Lead,Client Lead,active_no_providers,active_no_providers


In [36]:
# Doing a regex search to find string patterns

def find_paid_nonpaid(s):
    if re.search(r'non.*?paid', s, re.I):
        return 'non-paid'
    elif re.search(r'Buyerzone|^paid\s+', s, re.I):
        return 'paid'
    else:
        return 'unknown'

In [72]:
%%timeit
# our old friend .apply()
df['lead_source_paid_unpaid'] = df['Lead Source'].apply(find_paid_nonpaid)

480 ms ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


How does `np.vectorize()` do for strings?

In [74]:
%%timeit

vect_str = np.vectorize(find_paid_nonpaid)

df['lead_source_paid_unpaid1'] = vect_str(df['Lead Source'])

535 ms ± 9.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
%%timeit
# How does a list comprehension do?
df['lead_source_paid_unpaid2'] = ['non-paid' if re.search(r'non.*?paid', s, re.I) 
                                  else 'paid' if re.search(r'Buyerzone|^paid\s+', s, re.I) 
                                  else 'unknown' for s in df['Lead Source']]

524 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


pandas provides the `.str()` methods for working with strings.

In [76]:
%%timeit
# How does this compare?
conditions = [
    df['Lead Source'].str.contains(r'non.*?paid', case=False, na=False),
    df['Lead Source'].str.contains(r'Buyerzone|^paid\s+', case=False, na=False),
]

choices = [
    'non-paid',
    'paid'
]

df['lead_source_paid_unpaid1'] = np.select(conditions, choices, default='unknown')

493 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


*what about not searching strings?*

In [40]:
%%timeit
df['lowerls'] = df['Lead Source'].apply(lambda x: x.lower())

58 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
%%timeit
df['lowerls1'] = df['Lead Source'].str.lower()

69.9 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Dictionary lookups

In [77]:
channel_dict = {
    'Billing Service': 'BS', 'Consultant': 'PD', 'Educational': 'PD', 
    'Enterprise': 'PD', 'Hospital': 'PD', 'IPA': 'PD', 'MBS': 'RCM', 
    'MSO': 'PD', 'Medical practice': 'PD', 'Other': 'PD', 'Partner': 'PD',
    'PhyBillers': 'BS', 'Practice': 'PD', 'Purchasing Group': 'PD',
    'Reseller': 'BS', 'Vendor': 'PD', '_Other': 'PD', 'RCM': 'RCM'
}

def a_dict_lookup(row):
    if row['Providers'] > 7:
        return 'Upmarket'
    else:
        channel = channel_dict.get(row['Category'])
        return channel

In [78]:
%%timeit
df['dict_lookup'] = df.apply(a_dict_lookup, axis=1)

5.15 s ± 58.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
%%timeit
df['dict_lookup1'] = np.where(
    df['Providers'].values > 7, 
    'Upmarket',
    df['Category'].map(channel_dict)
)

/home/nathancheever/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


17.5 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [81]:
(5.15 * 1000) / 17.5

294.2857142857143

In [80]:
%%timeit
channel_values = df['Category'].map(channel_dict)
df['dict_lookup1'] = np.where(
    df['Providers'] > 7, 
    'Upmarket',
    channel_values
)

19.6 ms ± 452 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
%%timeit
# Using np.vectorize to vectorize a dictionary .get() method works, but is slower than .map()
channel_values = np.vectorize(channel_dict.get)(df['Category'].values)
df['dict_lookup2'] = np.where(
    df['Providers'] > 7, 
    'Upmarket',
    channel_values
)

37.7 ms ± 730 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
print((df['dict_lookup'] == df['dict_lookup1']).all())
print((df['dict_lookup'] == df['dict_lookup2']).all())

True
True


#### Dates

In [48]:
df.head(2)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,...,lead_source_paid_unpaid2,lead_source_paid_unpaid1,lowerls,lowerls1,dict_lookup,dict_lookup1,dict_lookup2
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,...,non-paid,non-paid,nonpaid : email : webinar : efficiency kiosk a...,nonpaid : email : webinar : efficiency kiosk a...,PD,PD,PD
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,...,non-paid,non-paid,nonpaid : email : webinar : efficiency kiosk a...,nonpaid : email : webinar : efficiency kiosk a...,PD,PD,PD


In [49]:
# make a new column called 'Start Date' for dummy testing
# ONly do a fraction so we have some NaN values
df['Start Date'] = df['Date Created'].sample(frac=0.8)

In [50]:
def weeks_to_complete(row) -> float:
    """Calculate the number of weeks between two dates"""
    if pd.isnull(row['Start Date']):
        return (row['Original Record: Date Created'] -  row['Date Created']).days / 7
    else:
        return (row['Date Created'] - row['Start Date']).days / 7

In [51]:
%%timeit
wtc1 = df.apply(weeks_to_complete, axis=1)

12.7 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


One approach to vectorization is to use pandas `.dt` datetime accessors. They have lots of goodies..

In [52]:
%%timeit
wtc2 = np.where(
    df['Start Date'].isnull().values,
    (df['Original Record: Date Created'].values - df['Date Created']).dt.days / 7,
    (df['Date Created'].values - df['Start Date']).dt.days / 7
)

18.6 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Another approach is to do ndarray type casting, converting our series into numpy timedelta arrays. This way is faster, but more verbose and kinda more code for basically the samething.

In [53]:
%%timeit
wtc3 = np.where(
    df['Start Date'].isnull().values,
    ((df['Original Record: Date Created'].values - df['Date Created'].values).astype('timedelta64[D]') / np.timedelta64(1, 'D')) / 7,
    ((df['Date Created'].values - df['Start Date'].values).astype('timedelta64[D]') / np.timedelta64(1, 'D')) / 7
)

7.91 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
# How much faster is our last way over .apply()?
(12.7 * 1000) / 7.91

1605.5625790139063

#### Needing values on other rows for the logic
This example comes from a task I had to recreate a function like this in Excel:
```excel
=IF(A2=A1, IF(L2-L1 < 5, 0, 1), 1))
```
Where the `A` column is for ids, and the `L` column is for dates.

In [55]:
def time_looper(df):
    """ Using a plain Python for loop"""
    output = []
    for i, row in enumerate(range(0, len(df))):
        if i > 0:
            
            # compare the current id to the row above
            if df.iloc[i]['Internal ID'] == df.iloc[i-1]['Internal ID']:
                
                # compare the current date to the row above
                if (df.iloc[i]['Date Created'] - df.iloc[i-1]['Original Record: Date Created']).days < 5:
                    output.append(0)
                else:
                    output.append(1)
            else:
                output.append(1)
        else:
            output.append(np.nan)
    return output

In [56]:
def time_looper2(df):
    """Using pandas dataframe `.iterrows()` method for iterating over rows"""
    output = []
    for i, row in df.iterrows():
        if i > 0:
            if df.iloc[i]['Internal ID'] == df.iloc[i-1]['Internal ID']:
                if (df.iloc[i]['Date Created'] - df.iloc[i-1]['Original Record: Date Created']).days < 5:
                    output.append(0)
                else:
                    output.append(1)
            else:
                output.append(1)
        else:
            output.append(np.nan)
    return output

In [57]:
df.sort_values(['Internal ID', 'Date Created'], inplace=True)

In [58]:
%%time
df['time_col_raw_for'] = time_looper(df)

CPU times: user 2min 1s, sys: 7.17 ms, total: 2min 1s
Wall time: 2min 1s


In [59]:
%%time
df['time_col_iterrows'] = time_looper2(df)

CPU times: user 2min 19s, sys: 67.5 ms, total: 2min 19s
Wall time: 2min 19s


Our approach to vectorizing this unfortunate situation was two-fold:
 1. Using the pandas `.shift()` function, we moved previous values down so they're on the same axis as what we're comparing them to 
 2. `np.select()` for the vectorized conditional logic check
 

In [60]:
%%timeit
previous_id = df['Internal ID'].shift(1).fillna(0).astype(int)
previous_date = df['Original Record: Date Created'].shift(1).fillna(pd.Timestamp('1900'))

conditions = [
    ((df['Internal ID'].values ==  previous_id) & 
     (df['Date Created'] - previous_date).astype('timedelta64[D]') < 5),
    df['Internal ID'].values ==  previous_id
]
choices = [0, 1]
df['time_col1'] = np.select(conditions, choices, default=1)

11.1 ms ± 49.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
(((2 * 60) + 19) * 1000) / 11.1

12522.522522522522

In [64]:
(df['time_col1'] == df['time_col_iterrows']).all()

False

# Other alternatives

## A _parallel_ apply func
Source: https://towardsdatascience.com/make-your-own-super-pandas-using-multiproc-1c04f41944a1

In [65]:
from multiprocessing import Pool

In [66]:
def p_apply(df, func, cores=4):
    """Pass in your dataframe and the func to apply to it"""
    df_split = np.array_split(df, cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [67]:
# df = p_apply(df, func=some_big_function)

## Dask
https://docs.dask.org